# <span style="color:blue"> MBA em Ciência de Dados</span>
# <span style="color:blue">Programação para Ciência de Dados</span>

## <span style="color:blue">Pandas Parte III - Avaliação </span>
**Material Produzido por Luis Gustavo Nonato**<br>
**Cemeai - ICMC/USP São Carlos**

Os exercícios a seguir farão uso do arquivo <font style="font-family: monaco">sales1000.csv</font> disponível para download no moodle.

O arquivo <font style="font-family: monaco">sales1000.csv</font> possui informações relativas à venda de produtos. As colunas abaixo são de particular importância para solução dos exercícios:
- *Country*: país da venda
- *Item Type*: tipo do produto vendido no país
- *Units Sold*: quantidade vendida
- *Unit Price*: preço de venda
- *Unit Cost*: preço de custo
- *Total Profit*: lucro total

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('sales1000.csv')

### Exercício 1)
Agrupe os elementos da base de dados armazenada no arquivo <font style="font-family: monaco">sales1000.csv</font> de acordo com os países.
Quantos produtos diferentes o país "Portugal" vende?

a) 7<br>
b) 6<br>
c) 5<br>
d) 4

In [94]:
df.shape


(1000, 14)

In [83]:
#Solucao
df_country = df.groupby(["Country"])
portugal = df_country.get_group('Portugal')

print(type(df_country))

print(portugal[['Item Type','Units Sold','Total Profit']])
print('\n Nr de produtos diferentes de Portugal == ',portugal['Item Type'].nunique())


<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
           Item Type  Units Sold  Total Profit
60   Office Supplies         949     119811.25
130  Office Supplies        8788    1109485.00
302  Office Supplies        9532    1203415.00
649       Vegetables        5511     347909.43
738        Cosmetics        4029     700522.23
999           Cereal        3528     312545.52

 Nr de produtos diferentes de Portugal ==  4


### Exercício 2)
Considerando todas as vendas de um país, qual o país com o maior lucro total?

a) Cameroon<br>
b) Japan<br>
c) Australia<br>
d) Cuba

In [7]:
# Solucao
print(type(df_country))
country_max_profit = df.groupby(['Country'])['Total Profit'].sum().max()

print(df.groupby(['Country']).filter(lambda x: x['Total Profit'].sum() >= country_max_profit))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
                                Region Country        Item Type Sales Channel  \
85   Central America and the Caribbean    Cuba        Beverages       Offline   
127  Central America and the Caribbean    Cuba          Clothes       Offline   
233  Central America and the Caribbean    Cuba  Office Supplies       Offline   
364  Central America and the Caribbean    Cuba        Household       Offline   
427  Central America and the Caribbean    Cuba             Meat        Online   
455  Central America and the Caribbean    Cuba    Personal Care        Online   
462  Central America and the Caribbean    Cuba        Cosmetics       Offline   
581  Central America and the Caribbean    Cuba           Cereal       Offline   
582  Central America and the Caribbean    Cuba        Household       Offline   
768  Central America and the Caribbean    Cuba  Office Supplies        Online   
918  Central America and the Caribbean    Cuba        

In [8]:
# Solucao

max_profit = df.groupby(['Country']).apply(lambda x: (x['Total Profit'].sum()))

mask = (max_profit >= max_profit.max())

print(max_profit[mask])

Country
Cuba    7351820.83
dtype: float64


### Exercício 3)
Quantos países vendem exatamente sete produtos distintos? 

a) 9<br>
b) 8<br>
c) 7<br>
d) 6

In [56]:
#Solucao
prod_dif = df.groupby(['Country'])['Item Type'].nunique()

mask = (prod_dif == 7)

print('Numero de paises com 7 produtos diferentes = ',prod_dif[mask].count())
print('\n',prod_dif[mask])

Numero de paises com 7 produtos diferentes =  9

 Country
Angola            7
Belgium           7
Czech Republic    7
Fiji              7
Iran              7
Romania           7
South Sudan       7
Turkey            7
Vanuatu           7
Name: Item Type, dtype: int64


### Exercício 4)
Qual o país com a maior diferença entre a maior quantidade e a menor quantidade de produtos vendidos? Por exemplo, 'Tonga' vende os seguintes quantidades de produtos:
```python
Household  8635
Beverages  9367
Meat       9633
Fruits     7124
Meat       3499
```
Portando, a diferença entre a maior e a menor quantidade de produtos vendidos é $$9633-3499=6134$$ Qual o país com a maior diferença?

a) Tonga<br>
b) Cape Verde<br>
c) Romania<br>
d) Cuba

In [113]:
id_mais_vendido = df.groupby('Country')['Units Sold'].transform(lambda x: x.idxmax())
id_menos_vendido = df.groupby('Country')['Units Sold'].transform(lambda x: x.idxmin())


paises = pd.DataFrame(df.groupby('Country').apply(lambda x:x.name).values,
                        columns=['Pais'])

aux_max = id_mais_vendido.groupby(df['Country'].values).apply(lambda i: i.iloc[0]).values
aux_min = id_menos_vendido.groupby(df['Country'].values).apply(lambda i: i.iloc[0]).values


paises['Prod mais vendido'] = df['Item Type'].iloc[aux_max].values
paises['Prod Menos vendido'] = df['Item Type'].iloc[aux_min].values
paises['Valor mais vendido'] = df['Units Sold'].iloc[aux_max].values
paises['Valor Menos vendido'] = df['Units Sold'].iloc[aux_min].values
paises['delta'] = df['Units Sold'].iloc[aux_max].values - df['Units Sold'].iloc[aux_min].values

print('Maior dif entre prod + e - vendido = ',paises['delta'].max(),'e o seu indice é = ',paises['delta'].idxmax())
print('\n',paises[['Pais','delta']].iloc[[paises['delta'].idxmax()]])

print('\n',paises)


Maior dif entre prod + e - vendido =  9775 e o seu indice é =  27

           Pais  delta
27  Cape Verde   9775

              Pais Prod mais vendido Prod Menos vendido  Valor mais vendido  \
0     Afghanistan            Cereal          Baby Food                7081   
1         Albania         Beverages          Baby Food                8775   
2         Algeria     Personal Care          Baby Food                9681   
3         Andorra     Personal Care          Beverages                9950   
4          Angola         Household          Beverages                9135   
..            ...               ...                ...                 ...   
180  Vatican City         Beverages         Vegetables                7946   
181       Vietnam            Snacks      Personal Care                7665   
182         Yemen            Cereal    Office Supplies                7073   
183        Zambia         Baby Food             Fruits                7526   
184      Zimbabwe           

### Exercício 5)
O lucro de um produto é a diferença entre o preço de venda e o de custo. Qual o produto que fornece o maior lucro médio?

a) Baby Food<br>
b) Fruits<br>
c) Household<br>
d) Cosmetics

In [ ]:
# Solucao